In [25]:
import copy
import networkx as nx
import sys
import os
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from numpy.linalg import inv,multi_dot
from scipy.linalg import expm
from matplotlib import pyplot as plt
from networkx.drawing.nx_pydot import graphviz_layout
import numpy as np
import pydot
from Kernels.Random_Walk import Random_Walk
from Kernels.Shortest_Path import Shortest_Path
from Kernels.Weisfeiler_Lehman import Weisfeiler_Lehman
#from Kernels.WL_Version1 import Weisfeiler_Lehman
from Kernels.WL_Transformer import WL_Transformer
from Kernels.WL_edge import WL_edge
from Kernels.WL_Shortest_Path import WL_Shortest_Path
from Kernels.three_node_graphlet_kernel import Three_node_graphlet_kernel
from Kernels.k_node_graphlet_kernel import k_node_graphlet_kernel
from Kernels.Shortest_Path_Labelled import Shortest_Path_Labelled
from Kernels.utils import hierarchy_pos,get_nxgraph_from_adjanecy_matrix,count_commons
from Datasets.utils_mutag import load_data , create_loaders 
import Datasets


In [27]:
import torch
from torch_geometric.nn import global_mean_pool
print(torch.__version__)
import torchnet as tnt
import torch.nn.functional as F
import torch.nn as nn
import wget
import zipfile

1.12.1+cpu


In [3]:
"""
# %%
wget.download(
    ")

file_name = 'MUTAG.zip'
zip_File = zipfile.ZipFile(file_name, 'r')
# 
zip_File.extractall()  

"""
# %%

'\n# %%\nwget.download(\n    ")\n\nfile_name = \'MUTAG.zip\'\nzip_File = zipfile.ZipFile(file_name, \'r\')\n# \nzip_File.extractall()  \n\n'

Node labels:

  0  C\
  1  N\
  2  O\
  3  F\
  4  I\
  5  Cl\
  6  Br\
\
Edge labels:\
\
  0  aromatic\
  1  single\
  2  double\
  3  triple\

In [29]:
dataset = load_data(path='Datasets/MUTAG/', ds_name='MUTAG',
                    use_node_labels=True, use_edge_labels=True, max_node_label=7,max_edge_label=4)


#pytorch loaders
train_dataset, val_dataset = create_loaders(
    dataset, batch_size=1, split_id=150, offset=0)
print('Data are ready')

../Datasets/MUTAG/MUTAG_graph_indicator.txt
Data are ready


In [5]:

X=np.array(dataset,dtype=object)[:,0]
y=np.array(dataset,dtype=object)[:,1]
G_train, G_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [6]:
dataset[0][0].nodes(data=True)

NodeDataView({1: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 2: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 3: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 4: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 5: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 6: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 7: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 8: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 9: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 10: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 11: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 12: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 13: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 14: {'attr_dict': array([1., 0., 0., 0., 0., 0., 0.])}, 15: {'attr_dict': array([0., 1., 0., 0., 0., 0., 0.])}, 16: {'attr_dict': array([0., 0., 1., 0., 0., 0., 0.])}, 17: {'attr_dict': array([0., 0., 1., 0., 0., 0., 0.])}})

In [7]:
dataset[0][0].edges(data=True)

EdgeDataView([(1, 2, {'edge_label': array([1., 0., 0., 0.])}), (1, 6, {'edge_label': array([1., 0., 0., 0.])}), (2, 3, {'edge_label': array([1., 0., 0., 0.])}), (3, 4, {'edge_label': array([1., 0., 0., 0.])}), (4, 5, {'edge_label': array([1., 0., 0., 0.])}), (4, 10, {'edge_label': array([1., 0., 0., 0.])}), (5, 6, {'edge_label': array([1., 0., 0., 0.])}), (5, 7, {'edge_label': array([1., 0., 0., 0.])}), (7, 8, {'edge_label': array([1., 0., 0., 0.])}), (8, 9, {'edge_label': array([1., 0., 0., 0.])}), (9, 10, {'edge_label': array([1., 0., 0., 0.])}), (9, 14, {'edge_label': array([1., 0., 0., 0.])}), (10, 11, {'edge_label': array([1., 0., 0., 0.])}), (11, 12, {'edge_label': array([1., 0., 0., 0.])}), (12, 13, {'edge_label': array([1., 0., 0., 0.])}), (13, 14, {'edge_label': array([1., 0., 0., 0.])}), (13, 15, {'edge_label': array([0., 1., 0., 0.])}), (15, 16, {'edge_label': array([0., 0., 1., 0.])}), (15, 17, {'edge_label': array([0., 1., 0., 0.])})])

In [23]:
updated=transform_labels_to_integers_from_a_list_of_graphs(G_train,node_label='attr_dict')
updated[0].nodes(data=True)

NodeDataView({2186: {'attr_dict': 3}, 2187: {'attr_dict': 2}, 2188: {'attr_dict': 3}, 2189: {'attr_dict': 3}, 2190: {'attr_dict': 3}, 2191: {'attr_dict': 3}, 2192: {'attr_dict': 3}, 2193: {'attr_dict': 3}, 2194: {'attr_dict': 3}, 2195: {'attr_dict': 2}, 2196: {'attr_dict': 2}, 2197: {'attr_dict': 1}, 2198: {'attr_dict': 1}})

In [15]:
#GN=[G1,G2,G3,G4]
kernel2=Weisfeiler_Lehman(normalise=True,h=0,node_label='attr_dict')
k_train=kernel2.fit_transform(list(G_train))
k_test=kernel2.transform(list(G_test))

In [ ]:
# Uses the SVM classifier to perform classification
clf = SVC(kernel='precomputed')
clf.fit(np.asarray(k_train), np.ravel(y_train).astype(int))
y_pred = clf.predict(np.asarray(k_test))

# Computes and prints the classification accuracy
acc = accuracy_score(np.array(y_test,dtype=int), np.array(y_pred,dtype=int))
print("Accuracy:", str(round(acc*100, 2)) + "%")

Accuracy: 63.16%


In [ ]:
# Uses the SVM classifier to perform classification
clf = SVC(kernel='precomputed')
clf.fit(np.asarray(k_train), np.ravel(y_train).astype(int))
y_pred = clf.predict(np.asarray(k_test))
# Computes and prints the classification accuracy
acc = accuracy_score(np.array(y_test,dtype=int), np.array(y_pred,dtype=int))
print("Accuracy:", str(round(acc*100, 2)) + "%")

Accuracy: 89.47%


In [ ]:
kernel=Shortest_Path(normalise=True)
k_train=kernel.fit_transform(list(G_train))
k_test=kernel.transform(list(G_test))
#dictionary=kernel.get_phi()

In [ ]:
# Uses the SVM classifier to perform classification
clf = SVC(kernel='precomputed')
clf.fit(np.asarray(k_train), np.ravel(y_train).astype(int))
y_pred = clf.predict(np.asarray(k_test))

# Computes and prints the classification accuracy
acc = accuracy_score(np.array(y_test,dtype=int), np.array(y_pred,dtype=int))
print("Accuracy:", str(round(acc*100, 2)) + "%")

Accuracy: 94.74%


In [11]:
kernel=Three_node_graphlet_kernel(normalise=True)
k_train=kernel.fit_transform(list(G_train))
k_test=kernel.transform(list(G_test))

# Uses the SVM classifier to perform classification
clf = SVC(kernel='precomputed')
clf.fit(np.asarray(k_train), np.ravel(y_train).astype(int))
y_pred = clf.predict(np.asarray(k_test))

# Computes and prints the classification accuracy
acc = accuracy_score(np.array(y_test,dtype=int), np.array(y_pred,dtype=int))
print("Accuracy:", str(round(acc*100, 2)) + "%")

Accuracy: 94.74%


(169, 169)

array([1, 1, -1, 1, 1, 1, 1, -1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1],
      dtype=object)